# 地方競馬（NAR）特徴量エンジニアリング分析 (Feature Engineering Analysis)

このノートブックでは、`NarFeatureGenerator` によって生成された新規特徴量（JIT特徴量）の品質と分布を確認します。
特に、**過去n戦**の指定が柔軟になった点や、データの連続性を確認します。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

# プロジェクトのsrcディレクトリをパスに追加
src_path = os.path.abspath(os.path.join(os.getcwd(), '../../src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from nar.loader import NarDataLoader
from nar.features import NarFeatureGenerator

%matplotlib inline
sns.set(font="IPAexGothic", style="whitegrid")

## 1. データのロードと汎用的な過去走特徴量の生成

`history_windows` を自由に指定して特徴量を生成します。ここでは [1, 2, 3, 5, 10] を指定してみます。

In [ ]:
loader = NarDataLoader()
raw_df = loader.load(limit=100000, history_start_date='2018-01-01') # より長い期間をロード

# 過去 1, 2, 3, 5, 10 戦の窓を指定
windows = [1, 2, 3, 5, 10]
feature_gen = NarFeatureGenerator(history_windows=windows)
df = feature_gen.generate_features(raw_df)

print(f"生成後の全カラム数: {len(df.columns)}")
history_cols = [col for col in df.columns if 'horse_prev' in col]
print(f"生成された過去走カラム数: {len(history_cols)}")
print(f"確認用カラムリスト: {history_cols[:10]} ...")

## 2. データの連続性とNaNの状況確認

`min_periods=1` の設定により、出走回数が窓サイズに満たない場合でも、存在する限りの過去走で計算されているかを確認します。

In [ ]:
# 出走回数別のNaNの数を確認
nan_check = df.groupby('horse_run_count')[['horse_prev1_rank_avg', 'horse_prev5_rank_avg', 'horse_prev10_rank_avg']].apply(lambda x: x.isnull().sum())
print("出走回数ごとの各特徴量の欠損数:")
display(nan_check.head(15))

## 3. 指定した過去n戦の着順相関の比較

「直近1戦」と「過去10戦平均」でどれほど今走の着順への感度が違うか比較します。

In [ ]:
compare_cols = ['horse_prev1_rank_avg', 'horse_prev3_rank_avg', 'horse_prev10_rank_avg']
valid_df = df.dropna(subset=['horse_prev10_rank_avg'])

plt.figure(figsize=(18, 5))
for i, col in enumerate(compare_cols):
    plt.subplot(1, 3, i+1)
    # 散布図と回帰直線
    sns.regplot(data=valid_df.sample(min(3000, len(valid_df))), x=col, y='rank', 
                scatter_kws={'alpha':0.2, 's':10}, line_kws={'color':'red'})
    plt.title(f"{col} vs 今走の着順")
    plt.ylim(0, 18)

plt.tight_layout()
plt.show()

## 4. 全特徴量の相関ランキング

指定したすべての窓サイズの平均着順と今走の着順の相関を網羅的に確認します。

In [ ]:
rank_avg_cols = [col for col in df.columns if '_rank_avg' in col]
corrs = df[['rank'] + rank_avg_cols].corr()['rank'].sort_values()

plt.figure(figsize=(10, 6))
corrs.drop('rank').plot(kind='barh', color='darkblue')
plt.title("過去n戦平均着順と今走着順の相関係数比較")
plt.grid(axis='x', linestyle='--')
plt.show()

## 5. 結論

- 引数 `history_windows` を通じて任意の過去走数を集計できるようになりました。
- `min_periods=1` により、キャリアの浅い馬でも「前走のみ」などのデータが保持され、情報が途切れにくい構造を確認しました。
- 窓サイズを広げる（1 -> 10）ほど、相関係数がどのように変化するかを定量的に確認できました。